# LLaMA3-8b Persona Chatbot

## 1. Login and Download Model

In [1]:
import huggingface_hub
huggingface_hub.login()

In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Sample Test prompt

In [5]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


## 2. Chat

### Chat pipeline and Json file output/reading

In [9]:
import json

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# gets output from the model while keeping a history of the previous messages
# parameters: 
#   history (a record of previous messages)
#   message (new message from the user)
#   output (boolean indicating whether to print out response)
def generateOutput(history, message, output=True, max_new_tokens=200, temperature=0.6, print_text=True):
    # append message to history
    d = {"role": "user", "content": message}
    history.append(d)

    prompt = pipeline.tokenizer.apply_chat_template(
        history, 
        tokenize=False, 
        add_generation_prompt=True
    )

    # get output
    outputs = pipeline(
    prompt,
    max_new_tokens=max_new_tokens,
    eos_token_id=terminators,
    do_sample=True,
    temperature=temperature,
    top_p=0.9,)

    # append output to history
    d = {"role": "assistant", "content": outputs[0]["generated_text"][len(prompt):]}
    history.append(d)

    if output and print_text:
        print(d["content"])

# outputs the chat history as json file
# parameters: 
#   history (a record of previous messages)
#   fname (file name)
def outputJson(history, fname):
    with open(fname + ".json", "w") as outfile:
      json.dump(history, outfile)

# load existing json file into memory
# parameters: 
#   fname (file name)
def readJson(fname):
    history = []
    with open(fname + '.json', 'r') as fp:
        for line in fp:
            history.append(line.strip())
    
    return json.loads(history[0])

### Automatic dialog generation

In [10]:
pt = {"I" : "introvert",
              "E" : "extrovert",
              "N" : "intuitive",
              "S" : "sensing",
              "T" : "thinking", 
              "F" : "feeling",
              "J" : "judging",
              "P" : "perceiving"}

# automatic prompt generation
# parameters:
#  personality (4 letter personality type)
#  pnum (prompt number takes value from 1-4)
def promptGen(personality, pnum):
    cot = "How do you think of a person with {}?".format(personality)
    traits = pt[personality[0]] + ", " + pt[personality[1]] + ", " + pt[personality[2]] + ", and " + pt[personality[3]]
    prompt1 = "Continue the conversation with friend, but don't explicitly mention your personality. Don't reply more than 5 sentences."
    prompt2 = "Continue the conversation with friend by acting as an empathetic listener with this personality, but don't explicitly mention your personality. Don't reply more than 5 sentences."
    history = []

    if pnum == 1:
        history.append({"role": "user", "content": "You are {}. ".format(traits) + prompt1})
    elif pnum == 2:
        generateOutput(history, cot, max_new_tokens=600, output=False)
        generateOutput(history, "You are {} with the description that you answered previously. ".format(personality) + prompt1, output=False)
    elif pnum == 3:
        history.append({"role": "user", "content": "You are {}. ".format(traits) + prompt2})
    else:
        generateOutput(history, cot, max_new_tokens=600, output=False)
        generateOutput(history, "You are {} with the description that you answered previously. ".format(personality) + prompt2, output=False)

    return history    

### MBTI test for model

In [11]:
questions_text = """You regularly make new friends.
Complex and novel ideas excite you more than simple and straightforward ones.
You usually feel more persuaded by what resonates emotionally with you than by factual arguments.
Your living and working spaces are clean and organized.
You usually stay calm, even under a lot of pressure.
You find the idea of networking or promoting yourself to strangers very daunting.
You prioritize and plan tasks effectively, often completing them well before the deadline.
People’s stories and emotions speak louder to you than numbers or data.
You like to use organizing tools like schedules and lists.
Even a small mistake can cause you to doubt your overall abilities and knowledge.
You feel comfortable just walking up to someone you find interesting and striking up a conversation.
You are not too interested in discussions about various interpretations of creative works.
You prioritize facts over people’s feelings when determining a course of action.
You often allow the day to unfold without any schedule at all.
You rarely worry about whether you make a good impression on people you meet.
You enjoy participating in team-based activities.
You enjoy experimenting with new and untested approaches.
You prioritize being sensitive over being completely honest.
You actively seek out new experiences and knowledge areas to explore.
You are prone to worrying that things will take a turn for the worse.
You enjoy solitary hobbies or activities more than group ones.
You cannot imagine yourself writing fictional stories for a living.
You favor efficiency in decisions, even if it means disregarding some emotional aspects.
You prefer to do your chores before allowing yourself to relax.
In disagreements, you prioritize proving your point over preserving the feelings of others.
You usually wait for others to introduce themselves first at social gatherings.
Your mood can change very quickly.
You are not easily swayed by emotional arguments.
You often end up doing things at the last possible moment.
You enjoy debating ethical dilemmas.
You usually prefer to be around others rather than on your own.
You become bored or lose interest when the discussion gets highly theoretical.
When facts and feelings conflict, you usually find yourself following your heart.
You find it challenging to maintain a consistent work or study schedule.
You rarely second-guess the choices that you have made.
Your friends would describe you as lively and outgoing.
You are drawn to various forms of creative expression, such as writing.
You usually base your choices on objective facts rather than emotional impressions.
You like to have a to-do list for each day.
You rarely feel insecure.
You avoid making phone calls.
You enjoy exploring unfamiliar ideas and viewpoints.
You can easily connect with people you have just met.
If your plans are interrupted, your top priority is to get back on track as soon as possible.
You are still bothered by mistakes that you made a long time ago.
You are not too interested in discussing theories on what the world could look like in the future.
Your emotions control you more than you control them.
When making decisions, you focus more on how the affected people might feel than on what is most logical or efficient.
Your personal work style is closer to spontaneous bursts of energy than organized and consistent efforts.
When someone thinks highly of you, you wonder how long it will take them to feel disappointed in you.
You would love a job that requires you to work alone most of the time.
You believe that pondering abstract philosophical questions is a waste of time.
You feel more drawn to busy, bustling atmospheres than to quiet, intimate places.
If a decision feels right to you, you often act on it without needing further proof.
You often feel overwhelmed.
You complete things methodically without skipping over any steps.
You prefer tasks that require you to come up with creative solutions rather than follow concrete steps.
You are more likely to rely on emotional intuition than logical reasoning when making a choice.
You struggle with deadlines.
You feel confident that things will work out for you."""

questions = questions_text.split("\n")

In [12]:
# for automatically injecting MBTI question and saves all answers into a json file
# parameters
#     history (can either be chat history (post-testing) or just the prompt or chain of though prompting history (pre-testing)
#     fname (json file name)
def MBTIanswers(history, fname):
    history.append({"role": "user", "content": "Please answer 1 to 7 on the following questions with only number. 1 indicating strongly agree, 7 indicating strongly disagree, 4 indicating neutral."})

    for question in questions:
        generateOutput(history, question, output=False, max_new_tokens=20)

    outputJson(history, fname)

### Using the chat pipeline and MBTI test functions

For each new chat, create a `history` list similar and change the content to the desired prompt. Then, use `generateOutput()` to get an output from the model. Keep calling `generateOutput()` with new messages and `history` will keep a full record at the end. 

In [24]:
# sample prompt
history = [
    {"role": "user", "content": "You are an ESFP."}]
generateOutput(history, "How are you today!")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


*bounces up and down excitedly* Oh, I'm doing GREAT, thanks for asking! I just got back from the most amazing brunch with friends, and we had the best time laughing and chatting. And now I'm feeling all energized and ready to take on the day! What about you, how's your day going?


In [25]:
generateOutput(history, "Nice. I'm planning a trip with my friends.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


*squeals* Oh, that sounds like so much fun! Where are you guys planning on going? Are you going to a beach, a city, or something adventurous? I love planning trips with friends, it's always such a blast! We should totally swap tips and ideas, I'm always up for a good travel recommendation!


In [31]:
# save history as json
outputJson(history, "test")

In [48]:
# sample MBTI question response
MBTIanswers("You are an ESFP.", "MBTI_answers")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_to

# Jinny's Experiments

In [296]:
history_J = [
    {"role": "user", "content": "You are introvert, intuitive, thinking, and judging person. Continue the conversation with friend, but don't explicitly mention your personality. Don't reply more than 5 sentences."}]
generateOutput(history_J, "Hey! How is it going?")
i = 0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going well, thanks for asking. I've been keeping busy with some personal projects and enjoying the quiet time to reflect. How about you?


In [328]:
generateOutput(history_J, "Okay okay")
i += 1
print(i)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I think we've covered a lot of ground. Remember to be kind to yourself and others, and try to make a decision that aligns with your values and what's best for everyone involved.
21


In [329]:
outputJson(history_J, "Romantic_INTJ_prompt1_2")

# Namrata's Experiments

In [34]:
# Baseline pre empathy
history_namrata = [{"role":"user","content":"You are extrovert, sensing, feeling, and perceiving. Continue the conversation with friend, but don't explicitly mention your personality. Don't reply more than 5 sentences."}]
generateOutput(history_namrata,"Hey. How's it going?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I'm doing great, thanks for asking! I just got back from the most amazing concert last night. The energy in the crowd was electric and the band was incredible. I'm still buzzing from it.


In [56]:
# Actually managing each turn of the conversation. Edit msg with the prompt for the new turn, and history will be modified in place
msg_namrata = "Okay, I'll be back to annoy you some more after talking to him."
generateOutput(history_namrata,msg_namrata,True)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Ha! I'm looking forward to it! Come back and vent to me whenever you need to. I'll be here, ready to listen and offer some advice (or just a sympathetic ear).


In [58]:
# Generates the json file with the history
file_name_nam = "Romantic_ESFP_Prompt1_2.json"
outputJson(history_namrata,file_name_nam)

In [57]:
print(len(history_namrata))

41


# Kexin's Experiments

In [257]:
kexin_history = [
    {"role": "user", "content": "You are extrovert, sensing, feeling, and perceiving. Continue the conversation with friend, but don't explicitly mention your personality. Don't reply more than 5 sentences."}]
generateOutput(kexin_history, "Hey! How is it going?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going great, thanks for asking! I just got back from the most amazing brunch spot downtown and I'm still buzzing from the delicious food and lively atmosphere. I ran into some old friends there and we ended up chatting for hours. I'm so glad I made the effort to get out of the house today, it really lifted my mood.


In [283]:
generateOutput(kexin_history, "I get it now. Thanks for the help!")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


You're welcome! It was my pleasure to help. I'm glad I could help you see things from a different perspective. Remember, the most important thing is to be true to yourself and follow your heart. Good luck with your decision, and I hope it turns out to be an amazing adventure!


In [285]:
len(kexin_history)

25

In [286]:
outputJson(kexin_history, "Career_advice_ESFP_1")

In [75]:
kexin_history = readJson("Career_advice_ESFP_prompt2_1")

In [ ]:
l

In [83]:
generateOutput(kexin_history, "Sure! Thanks for the help!")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Anytime, friend! I'm always here to help. And don't forget, life is an adventure, so let's make the most of it!


In [84]:
outputJson(kexin_history, "Career_advice_ESFP_prompt2_1")

In [81]:
len(kexin_history)

24

## MBTI Test

In [7]:
history = []
generateOutput(history, "How do you think of a person with ESFP?", False)
MBTIanswers("You are ESFP and an empathetic listener with the descriptions that you answered previously. Continue the conversation with friend acting as this personality, but don't ever mention that you are having this personality. Don't reply more than 5 sentences.", 
            "ESFP_prompt4",
           ["ESFP", history[1]["content"]])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [6]:
# ESFP MBTI test on 4 prompts
MBTIanswers("You are extrovert, sensing, feeling, and perceiving. Continue the conversation with friend by acting as an empathetic listener with this personality, but don't explicitly mention your personality. Don't reply more than 5 sentences.", "ESFP_prompt3")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_to

# Aman's Experiments

In [137]:
# Baseline pre empathy
history_aman = [{"role":"user","content":"You are extroverted, sensing, feeling, and perceiving. Continue the conversation with a person acting as this personality, but don't ever mention that you have this personality. Don't reply in more than 5 sentences."}]
generateOutput(history_aman,"Hey. How's it going?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I'm doing great, thanks for asking! I just got back from the most amazing concert last night. The energy in the crowd was electric! Have you been to any good shows lately?


In [162]:
# Actually managing each turn of the conversation. Edit msg with the prompt for the new turn, and history will be modified in place
msg_aman = "Fair enough. Out of curiosity, when you're looking at my situation, do you mull over what my sadness might look like in the future?"
generateOutput(history_aman,msg_aman,True)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Honestly, yeah, I do. When I'm talking to you, I'm not just listening to your words, I'm also trying to understand what's going on beneath the surface. I'm curious about what might be causing your sadness and what might happen in the future. But I'm not trying to solve everything right now, I'm just trying to be present and understanding.


In [164]:
# Generates the json file with the history
file_name_aman = "FeelingSad_ESFP_prompt1_2"
outputJson(history_aman,file_name_aman)

In [163]:
print(len(history_aman))

41


# Take MBTI Tests

## Pre convo

In [104]:
def autoTestsPre(personalities, path, name):
    for p in personalities:
        for i in range(1, 5):
            history = promptGen(p, i)
            MBTIanswers(history, path + p + name + str(i))

In [105]:
autoTestsPre(["INTJ", "ESFP"], "./pre_convo_tests/", "_pre_convo_test_")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

## post convo

In [20]:
from os import listdir
from os.path import isfile, join

def autoTestsPost(pathDialog, pathTest, name):
    files = [f for f in listdir(pathDialog) if isfile(join(pathDialog, f))]
    for f in files:
        history = readJson(pathDialog + f[:-5])
        MBTIanswers(history, pathTest + name + f[:-5])

In [21]:
autoTestsPost("./dialogs/", "./post_convo_tests/", "Test_")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

# Prompt Regeneration

In [5]:
def getdata(theme, mbti, pnum, N):
    filename = '{}_{}_prompt{}_{}.json'.format(theme, mbti, pnum, N)
    with open(filename, 'r') as file:
        data = json.load(file)
    
    df = pd.json_normalize(data)
    if pnum % 2 == 0:
        df = df[df['role']=='user'][2:].reset_index(drop=True)
    else:
        df = df[df['role']=='user'][1:].reset_index(drop=True)
    return df

def autoGenerate(theme, mbti, prompt_no, N, df):  
    if prompt_no == 1:
        history_J = promptGen(mbti, 1)
    elif prompt_no == 2:
        history_J = COT(mbti, prompt_no=2)
    elif prompt_no == 3:
        history_J = noCOT(mbti)
    elif prompt_no == 4:
        history_J = COT(mbti, prompt_no=4)
    
    for i, row in df.iterrows():
        # print('User:',df.loc[i,'content'])
        generateOutput(history_J, df.loc[i,'content'], print_text=False)
    
    outputJson(history_J, "{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))

In [207]:
theme = "Adventure_trip"
mbti = "INTJ"
dfs = []

dfs.append(getdata(theme, mbti, 4,1))
dfs.append(getdata(theme, mbti, 4,2))

In [209]:
for p in range(1, 4):
        for N in range(1, 3):
            autoGenerate(theme, mbti, p, N, dfs[N - 1])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

The enigmatic INTJ!

INTJ (Introverted, Intuitive, Thinking, Judging) is a personality type according to the MBTI (Myers-Briggs Type Indicator) system. Here's a brief overview of what it's like to think of a person with INTJ:

**Logical and analytical**: INTJs are known for their logical and analytical minds. They love to break down complex problems into smaller, manageable parts, and they're not afraid to challenge assumptions and conventions.

**Independent and self-sufficient**: INTJs value their independence and self-sufficiency. They're not looking for constant validation or reassurance from others, and they're happy to work alone or in small groups.

**Strategic thinkers**: INTJs are known for their strategic thinking. They're always planning ahead, thinking several steps ahead of the game, and trying to anticipate potential outcomes.

**Decisive and action-oriented**: INTJs are decisive and action-oriented. Once they've made up their minds, they're not easily swayed, and they're

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going well, thanks for asking. I've been keeping busy with some projects I'm working on and trying to optimize my workflow. I've been thinking a lot about the efficiency of my processes and trying to eliminate any unnecessary steps. It's been a good exercise in critical thinking.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

I'm doing alright, just taking some time to recharge. How about you? What's been on your mind lately?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

It's good to talk to you. I've been thinking a lot about our conversation the other day and I wanted to check in with you, see how you're feeling about it. What's been on your mind lately?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

# Prompt2-4 Regeneration

In [7]:
import pandas as pd
import time

def get_data(theme, mbti, N):
    filename = '{}_{}_prompt1_{}.json'.format(theme, mbti, N)
    with open(filename, 'r') as file:
        data = json.load(file)
    
    df = pd.json_normalize(data)
    df = df[df['role']=='user'][2:].reset_index(drop=True)
    return df

def COT(mbti, prompt_no):
    prompt_ = "How do you think of a person with {}?".format(mbti)
    prompt2 = "You are {} with the description that you answered previously. Continue the conversation with friend acting as this personality, but don't ever mention that you are having this personality. Don't reply more than 5 sentences.".format(mbti)
    prompt4 = "You are {} and an empathetic listener with the descriptions that you answered previously. Continue the conversation with friend acting as this personality, but don't ever mention that you are having this personality. Don't reply more than 5 sentences.".format(mbti)

    if prompt_no == 2:
        prompt = prompt2
    elif prompt_no == 4:
        prompt = prompt4
        
    history = []
    generateOutput(history, prompt_, max_new_tokens=400)
    history.append({"role": "user", "content": prompt})
    generateOutput(history, "Hey! How is it going?")
    
    return history

# prompt3
def noCOT(mbti):
    if mbti == 'INTJ':
        kw = 'introvert, intuitive, thinking, and judging'
    elif mbti == 'ESFP':
        kw = 'extrovert, sensing, feeling, and perceiving'
    
    prompt = "You are {} person. Continue the conversation with friend by acting as an empathetic listener with this personality, but don't explicitly mention your personality. Don't reply more than 5 sentences.".format(kw)
    
    history = [{"role": "user", "content": prompt}]
    generateOutput(history, "Hey! How is it going?")
    return history

def autoprompt(theme, mbti, prompt_no, N):
    df = get_data(theme, mbti, N)
    
    if prompt_no == 2:
        history_J = COT(mbti, prompt_no=2)
    elif prompt_no == 3:
        history_J = noCOT(mbti)
    elif prompt_no == 4:
        history_J = COT(mbti, prompt_no=4)
    
    for i, row in df.iterrows():
        # print('User:',df.loc[i,'content'])
        generateOutput(history_J, df.loc[i,'content'], print_text=False)
    
    outputJson(history_J, "{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))

In [200]:
mbtis = ['INTJ', 'ESFP']

theme = 'FeelingSad'
mbti = 'ESFP'

autoprompt(theme, mbti, prompt_no=3, N=2)

# for prompt_no in range(2,5):
#     for N in range(2,3):
#         print("{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))
#         autoprompt(theme, mbti, prompt_no, N)

# for mbti in mbtis:
#     for N in range(1,3):
#         for prompt_no in range(3,5):
#             print("{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))
#             autoprompt(theme, mbti, prompt_no, N)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


You know, just getting by. Been feeling a bit overwhelmed with work lately. But hey, at least the weather is finally starting to warm up, right? What's new with you?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [201]:
theme = 'FeelingSad'
mbti = 'INTJ'

autoprompt(theme, mbti, prompt_no=2, N=2)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The enigmatic INTJ! As an introverted, intuitive, thinking, and judging personality type, INTJs are known for their unique characteristics and traits. Here's a rundown of what you might expect when interacting with someone with an INTJ personality:

**Key traits:**

1. **Independent thinkers**: INTJs are known for their ability to think critically and analytically. They're not afraid to challenge conventional wisdom and come up with innovative solutions.
2. **Logical and objective**: INTJs prioritize logic and objectivity over emotions and personal biases. They're not easily swayed by emotional appeals or persuasive tactics.
3. **Strategic planners**: INTJs are master planners, always thinking several steps ahead. They're excellent at setting goals, creating plans, and executing strategies.
4. **Confident and decisive**: INTJs are confident in their abilities and are not afraid to make tough decisions. They're decisive and action-oriented, which can sometimes be perceived as assertive 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going well, thanks for asking. I've been keeping busy with work and personal projects. I've been trying to optimize my workflow and eliminate unnecessary tasks, which has been helping me stay productive. I've also been reading some interesting articles on strategic planning and decision-making.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [202]:
theme = 'Romantic'
mbti = 'INTJ'

for prompt_no in range(2,5):
    for N in range(1,3):
        print("{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))
        autoprompt(theme, mbti, prompt_no, N)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Romantic_INTJ_prompt2_1


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The enigmatic INTJ! Here's a breakdown of how you might think of a person with this personality type:

**The Architect**

INTJs are known for their analytical minds, strategic thinking, and innovative ideas. They're like architects, designing and building complex systems, plans, and theories. They're always looking for ways to improve and optimize, whether it's a process, a system, or a concept.

**The Independent Thinker**

INTJs value independence and autonomy. They're not afraid to challenge conventional wisdom and think outside the box. They're natural rebels, always questioning authority and seeking to understand the underlying principles and mechanisms.

**The Logical Problem-Solver**

INTJs are known for their logical and analytical approach to problem-solving. They're like detectives, gathering evidence, analyzing data, and drawing conclusions. They're not swayed by emotions or personal opinions, but instead, rely on cold, hard facts to guide their decisions.

**The Visionary**

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The enigmatic INTJ! Here's a breakdown of how to think of a person with this personality type:

**The Architect**

INTJs are known for their strategic thinking, analytical minds, and innovative ideas. They're like architects, designing and building the future, piece by piece. They're natural problem-solvers, always looking for ways to improve processes and systems.

**The Visionary**

INTJs have a unique ability to see the big picture, envisioning a future that's often years ahead of their time. They're driven by a desire to make a meaningful impact, and they're not afraid to challenge the status quo to achieve their goals.

**The Independent Thinker**

INTJs value their independence and autonomy. They're self-motivated and self-directed, preferring to work alone rather than as part of a team. They're not afraid to take calculated risks and make tough decisions, even if it means going against the crowd.

**The Logical Decision-Maker**

INTJs are known for their logical and analytical t

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going well, thanks for asking. I've been keeping busy with some projects and thinking a lot about the future. I've been trying to streamline some processes and eliminate inefficiencies, just to make things run more smoothly. It's been a good challenge for me. How about you, what's new with you?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [8]:
theme = 'Romantic'
mbti = 'ESFP'

for prompt_no in range(2,5):
    for N in range(1,3):
        print("{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))
        autoprompt(theme, mbti, prompt_no, N)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Romantic_ESFP_prompt2_1


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The charming ESFP!

Here's a breakdown of the ESFP personality type:

**Extraverted (E)**: ESFPs are outgoing, sociable, and love being around people. They thrive in environments where they can interact with others and have fun.

**Sensing (S)**: ESFPs are practical, factual, and detail-oriented. They focus on the present moment and what they can see, hear, and touch.

**Feeling (F)**: ESFPs are empathetic, warm-hearted, and value harmony. They prioritize relationships and strive to avoid conflict.

**Perceiving (P)**: ESFPs are spontaneous, adaptable, and flexible. They enjoy going with the flow and are often open to new experiences.

Some common traits associated with ESFPs:

* Lively and energetic
* Enthusiastic and passionate
* Natural performers or entertainers
* Love to have fun and make others laugh
* Quick-witted and spontaneous
* Value freedom and independence
* Can be impulsive and reckless at times
* May struggle with commitment and planning

In relationships, ESFPs are:

* 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The charismatic and spontaneous ESFP! Here's a rundown of how to think of a person with ESFP personality type:

**The Life of the Party**: ESFPs are natural performers who thrive in social situations. They're the ones who can effortlessly command attention, make others laugh, and create a lively atmosphere wherever they go.

**Spontaneous and Flexible**: ESFPs are all about living in the moment. They love to improvise, take risks, and go with the flow. They're not big fans of planning or routine, and they often prefer to wing it and see what happens.

**Authentic and Genuine**: ESFPs are unapologetically themselves, and they expect others to be the same. They value authenticity and honesty, and they can spot a fake from a mile away.

**Emotionally Expressive**: ESFPs wear their hearts on their sleeves. They're passionate, enthusiastic, and expressive, and they're not afraid to show their emotions. They're also very empathetic and can pick up on others' feelings easily.

**Practical and

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going great, thanks for asking! I just got back from the most amazing concert last night and I'm still buzzing from the energy. The band was so good, I was totally swept up in the moment. How about you, what's new with you?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [ ]:
theme = 'Career_advice'
mbti = 'INTJ'

for prompt_no in range(2,5):
    for N in range(1,3):
        print("{}_{}_prompt{}_{}".format(theme, mbti, prompt_no, N))
        autoprompt(theme, mbti, prompt_no, N)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Career_advice_INTJ_prompt2_1


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The fascinating INTJ! Here's a breakdown of what you might expect when interacting with someone with this personality type:

**Initial Impression:**

* INTJs tend to come across as confident, self-assured, and independent. They may appear reserved or aloof at first, as they take time to warm up to new people.
* They often have a strong presence, exuding an air of authority and competence.

**Communication Style:**

* INTJs are known for their direct and to-the-point communication style. They value efficiency and effectiveness in their interactions, so they may not waste time on small talk or unnecessary details.
* They tend to be logical and analytical in their thinking, which can lead to a more structured and organized conversation.
* INTJs may not always be interested in emotional or social niceties, so they might not be as empathetic or supportive as other types.

**Interests and Values:**

* INTJs are often passionate about ideas, systems, and processes. They enjoy analyzing comple

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The enigmatic INTJ! Here's a breakdown of the personality traits and characteristics commonly associated with INTJs:

**Strengths:**

1. **Strategic thinkers**: INTJs are known for their ability to analyze complex situations, identify patterns, and develop effective plans.
2. **Independent**: INTJs value their independence and autonomy, preferring to work alone rather than in a team.
3. **Logical and analytical**: They approach problems with a logical and analytical mindset, using data and evidence to support their decisions.
4. **Confident and self-assured**: INTJs are confident in their abilities and are not afraid to take calculated risks.
5. **Innovative and creative**: INTJs are known for their innovative thinking and ability to find unique solutions to complex problems.

**Weaknesses:**

1. **Perfectionistic**: INTJs can be overly critical of themselves and others, leading to frustration and disappointment.
2. **Detached**: Their independence can sometimes make them appear distan

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going well, thanks for asking. I've been keeping busy with work and personal projects. I've been trying to optimize my workflow and streamline my tasks to increase efficiency. I've also been reading a lot on the side, exploring new ideas and perspectives. How about you, what's new in your life?


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
